<a href="https://colab.research.google.com/github/daryoush/CudaAndFluxExperiments/blob/main/cudaexpOnJulia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### NOTE THIS WORKS AS PYTHON, IT ONLY WORKS WHEN THE RUNTIME COMPLAINS ABOUT JULIA RUNTIME, SO IT REVERTS TO PYTHON. AFTER JULIA IS INSTALLED, CHANGE TO JULIA RUNTIME

!curl -sSL "https://julialang-s3.julialang.org/bin/linux/x64/1.5/julia-1.5.0-linux-x86_64.tar.gz" -o julia.tar.gz
!tar -xzf julia.tar.gz -C /usr --strip-components 1
!rm -rf julia.tar.gz*
!julia -e 'using Pkg; pkg"add IJulia; precompile"'
!julia -e 'using Pkg; Pkg.add([ "CUDA", "Flux","Zygote","Profile", "TensorBoardLogger", "Logging",\
    "Random", "DataStructures", "Transformers","Statistics", "BenchmarkTools"\
    ]);' ###Pkg.precompile()'
!echo "DONE"

 Installing known registries into `~/.julia`
######################################################################## 100.0%
      Added registry `General` to `~/.julia/registries/General`
  Resolving package versions...
  Installed ZMQ ───────────── v1.2.1
  Installed ZeroMQ_jll ────── v4.3.2+5
  Installed Artifacts ─────── v1.3.0
  Installed VersionParsing ── v1.2.0
  Installed Conda ─────────── v1.5.0
  Installed SoftGlobalScope ─ v1.1.0
  Installed IJulia ────────── v1.23.1
  Installed JSON ──────────── v0.21.1
  Installed JLLWrappers ───── v1.1.3
  Installed MbedTLS ───────── v1.0.3
  Installed MbedTLS_jll ───── v2.16.8+1
  Installed Parsers ───────── v1.0.15
######################################################################## 100.0%
######################################################################## 100.0%
Updating `~/.julia/environments/v1.5/Project.toml`
  [7073ff75] + IJulia v1.23.1
Updating `~/.julia/environments/v1.5/Manifest.toml`
  [56f22d72] + Artifacts v1.3.0
  

In [1]:
[1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [2]:
using CUDA

┌ Info: Precompiling CUDA [052768ef-5323-5732-b1bb-66c8b64840ba]
└ @ Base loading.jl:1278


In [3]:
using BenchmarkTools

M = rand(2048, 2048)
@benchmark M^2

if ENV["COLAB_GPU"] == "1"

    M_gpu = cu(M)
    @benchmark CUDA.@sync M_gpu^2
else
    println("No GPU found.")
end

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1278


######################################################################### 100.0%


######################################################################### 100.0%


######################################################################### 100.0%


BenchmarkTools.Trial: 
  memory estimate:  416 bytes
  allocs estimate:  11
  --------------
  minimum time:     2.511 ms (0.00% GC)
  median time:      3.860 ms (0.00% GC)
  mean time:        3.886 ms (0.03% GC)
  maximum time:     8.792 ms (18.82% GC)
  --------------
  samples:          1286
  evals/sample:     1

In [12]:
CUDA.memory_status()
GC.gc(true)
CUDA.memory_status()
m=nothing

Effective GPU memory usage: 1.27% (191.875 MiB/14.726 GiB)
CUDA allocator usage: 16.051 MiB
binned usage: 16.051 MiB (16.051 MiB allocated, 0 bytes cached)
Effective GPU memory usage: 1.27% (191.875 MiB/14.726 GiB)
CUDA allocator usage: 16.051 MiB
binned usage: 16.051 MiB (16.016 MiB allocated, 36.000 KiB cached)


In [16]:
#using cuda arrays
a = CUDA.zeros(1024)
b = CUDA.ones(1024)
@show a.^2 .+ sin.(b)
CUDA.memory_status()





a .^ 2 .+ sin.(b) = Float32[0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.841471, 0.

In [24]:
a = CUDA.zeros(1024)

function kernel(a)
    i = threadIdx().x
    a[i] += 1
    return
end

@cuda threads=length(a) kernel(a)
@show a
CUDA.memory_status()

a = Float32[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0

In [19]:
for device in CUDA.devices()
    @show capability(device)
end

capability(device) = v"7.5.0"


In [25]:
a = CuArray([1])
a[1] += 1

┌ Warning: Performing scalar operations on GPU arrays: This is very slow, consider disallowing these operations with `allowscalar(false)`
└ @ GPUArrays /root/.julia/packages/GPUArrays/uaFZh/src/host/indexing.jl:43


2

In [30]:
using Statistics
a = CuArray([1,2])
@show var(a)
@show var(a, dims=1)  


var(a) = 0.5
var(a, dims = 1) = [0.5]


1-element CuArray{Float64,1}:
 0.5

In [31]:
a = CuArray{Float32}(undef, (1,2));
@show a .= 5
@show map(sin, a)

a .= 5 = Float32[5.0 5.0]
map(sin, a) = Float32[-0.9589243 -0.9589243]


1×2 CuArray{Float32,2}:
 -0.958924  -0.958924

In [32]:
CUDA.ones(2,3)
@show reduce(+, a)
@show mapreduce(sin, *, a; dims=2)
b = CUDA.zeros(1)
@show Base.mapreducedim!(identity, +, b, a)
a = CUDA.ones(2,3)
@show accumulate(+, a; dims=2)


reduce(+, a) = 10.0f0
mapreduce(sin, *, a; dims = 2) = Float32[0.9195358]
Base.mapreducedim!(identity, +, b, a) = Float32[10.0]
accumulate(+, a; dims = 2) = Float32[1.0 2.0 3.0; 1.0 2.0 3.0]


2×3 CuArray{Float32,2}:
 1.0  2.0  3.0
 1.0  2.0  3.0

In [34]:
a = CuArray([11,12,13])
@show findall(isodd, a)
@show findfirst(isodd, a)
b = CuArray([11 12 13; 21 22 23])
@show findmin(b)
@show findmax(b; dims=2)


findall(isodd, a) = [1, 3]
findfirst(isodd, a) = 1
findmin(b) = (11, CartesianIndex(1, 1))
findmax(b; dims = 2) = ([13; 23], CartesianIndex{2}[CartesianIndex(1, 3); CartesianIndex(2, 3)])


([13; 23], CartesianIndex{2}[CartesianIndex(1, 3); CartesianIndex(2, 3)])

In [35]:
# enable logging to demonstrate a CUBLAS kernel is used
CUBLAS.cublasLoggerConfigure(1, 0, 1, C_NULL)

 CUDA.rand(2,2) * CUDA.rand(2,2)

I! cuBLAS (v10.1) function cublasStatus_t cublasSgemm_v2(cublasContext*, cublasOperation_t, cublasOperation_t, int, int, int, const float*, const float*, int, const float*, int, const float*, float*, int) called:
i!  handle: type=cublasHandle_t; val=POINTER (IN HEX:0x0x13bfe000)
i!  transa: type=cublasOperation_t; val=CUBLAS_OP_N(0)
i!  transb: type=cublasOperation_t; val=CUBLAS_OP_N(0)
i!  m: type=int; val=2
i!  n: type=int; val=2
i!  k: type=int; val=2
i!  alpha: type=float; val=POINTER (IN HEX:0x0x7f31d5a0d670)
i!  A: type=float; val=POINTER (IN HEX:0x0x7f313262aa00)
i!  lda: type=int; val=2
i!  B: type=float; val=POINTER (IN HEX:0x0x7f313262e400)
i!  ldb: type=int; val=2
i!  beta: type=float; val=POINTER (IN HEX:0x0x7f31d5a0d6d0)
i!  C: type=float; val=POINTER (IN HEX:0x0x7f313265fa00)
i!  ldc: type=int; val=2
i! Time: 2020-12-25T20:10:35 elapsed from start 20.083333 minutes or 1205.000000 seconds
i!Process=727; Thread=139852844339840; GPU=0; Handle=POINTER (IN HEX:0x0x13bfe000); S

2×2 CuArray{Float32,2}:
 0.37541   0.0397653
 0.620094  0.0827805

In [37]:
#Certain operations, like the above matrix-matrix multiplication, also have a native 
#fallback written in Julia for the purpose of working with types that are not supported by CUBLAS:

# enable logging to demonstrate no CUBLAS kernel is used
CUDA.rand(Int128, 2, 2) * CUDA.rand(Int128, 2, 2)

2×2 CuArray{Int128,2}:
  55427753768220976152040712033798120442  …    5295419569899909331534571858596495689
 -40899183375511121400906318863278563568     106426877250577447098252310911788564515

In [38]:
x = CUDA.rand(2)
y = CUDA.rand(2)
@show CUBLAS.dot(2, x, y)
using LinearAlgebra
@show dot(Array(x), Array(y))


LoadError: ignored

In [39]:
using LinearAlgebra

 a = CUDA.rand(2,2)
2@show a = a * a'
@show cholesky(a)
a = CUDA.rand(2,2)
b = CUDA.rand(2,2)
@show a \ b
@show Array(a) \ Array(b)


a = a * a' = Float32[1.5849574 0.6022951; 0.6022951 0.49558416]
cholesky(a) = 

I! cuBLAS (v10.1) function cublasStatus_t cublasSgemm_v2(cublasContext*, cublasOperation_t, cublasOperation_t, int, int, int, const float*, const float*, int, const float*, int, const float*, float*, int) called:
i!  handle: type=cublasHandle_t; val=POINTER (IN HEX:0x0x13bfe000)
i!  transa: type=cublasOperation_t; val=CUBLAS_OP_N(0)
i!  transb: type=cublasOperation_t; val=CUBLAS_OP_T(1)
i!  m: type=int; val=2
i!  n: type=int; val=2
i!  k: type=int; val=2
i!  alpha: type=float; val=POINTER (IN HEX:0x0x7f31d08d3b30)
i!  A: type=float; val=POINTER (IN HEX:0x0x7f313262e400)
i!  lda: type=int; val=2
i!  B: type=float; val=POINTER (IN HEX:0x0x7f313262e400)
i!  ldb: type=int; val=2
i!  beta: type=float; val=POINTER (IN HEX:0x0x7f31d08d3b90)
i!  C: type=float; val=POINTER (IN HEX:0x0x7f3132626a00)
i!  ldc: type=int; val=2
i! Time: 2020-12-25T20:15:02 elapsed from start 24.533333 minutes or 1472.000000 seconds
i!Process=727; Thread=139852844339840; GPU=0; Handle=POINTER (IN HEX:0x0x13bfe000); S

Cholesky{Float32,CuArray{Float32,2}}(Float32[1.258951 0.47841033; 0.6022951 0.51643753], 'U', 0)
a \ b = Float32[1.540999 1.1794815; -0.80033207 0.059432443]
Array(a) \ Array(b) = Float32[1.540999 1.1794815; -0.80033207 0.059432436]


2×2 Array{Float32,2}:
  1.541     1.17948
 -0.800332  0.0594324

In [40]:
a = CUDA.rand(2,2)
using CUDA.CUFFT
fft(a)


2×2 CuArray{Complex{Float32},2}:
    2.16756+0.0im  0.0209978+0.0im
 -0.0281658+0.0im  -0.167229+0.0im